## Add custom preprocess layer to TF model

In [1]:
import json 
import tensorflow as tf

In [2]:
import numpy as np

In [5]:
path = "./output.jpg"

In [4]:
# this makes the cat_bytes to an array instead of a byte string 
cat_bytes = tf.expand_dims(tf.io.read_file(path), axis=0)

In [5]:
num_bytes = tf.io.read_file(path)

In [6]:
preprocess_single_image(num_bytes)

NameError: name 'preprocess_single_image' is not defined

In [49]:
def preprocess_single_image_tutorial(image_bytes, h=28, w=28):
    image = tf.image.decode_jpeg(image_bytes[0], channels=1)
    image = tf.image.resize(image, size=[h, w])
    image = (image - 127.5) / 127.5
    image = tf.expand_dims(image, axis=0)
    return image

In [51]:
image_bytes = tf.keras.Input(shape=[], batch_size=1, name='image_bytes', dtype=tf.string)
preprocessed_image = preprocess_single_image_tutorial(image_bytes)
# load model
model = tf.keras.models.load_model('./saved_model/my_model')
predictions = model(preprocessed_image)
new_model = tf.keras.Model(image_bytes, predictions)

# save model
new_model.save('export/1', save_format='tf')
print('Model Input Shape:', new_model.input_shape)

### !wget -q -O "cat.jpg" "https://images.pexels.com/photos/617278/pexels-photo-617278.jpeg?cs=srgb&dl=adorable-animal-blur-cat-617278.jpg&fm=jpg"

# print(tf.keras.applications.xception.decode_predictions(pred

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: export/1/assets
Model Input Shape: (1,)


In [62]:
loaded_model = tf.saved_model.load('export/1')
num_bytes = tf.expand_dims(tf.io.read_file(path), axis=0)
preds = loaded_model(num_bytes).numpy()

In [76]:
preds

array([[6.0136663e-14, 4.4565773e-13, 1.5706302e-01, 4.8600603e-02,
        1.5561676e-10, 2.6940592e-09, 1.1919244e-14, 2.4404729e-04,
        7.9409236e-01, 5.0815005e-13]], dtype=float32)

In [69]:
np.argmax(preds, axis=1)

array([8])

## My model

In [3]:
def preprocess_single_image(image_bytes):
    image = tf.image.decode_jpeg(image_bytes, channels=1)
    image = tf.image.resize(image, [28, 28])
    image = (255 - image) / 255.0  # normalize to [0,1] range
    image = tf.reshape(image, (1, 28, 28, 1))
    return image

In [6]:
num_bytes = tf.io.read_file(path)

In [7]:
image = preprocess_single_image(num_bytes)

In [8]:
image_np = image.numpy()

In [21]:
model = tf.keras.models.load_model('./saved_model/my_model')


In [13]:
probabilites = model.predict(image)
prediction = np.argmax(probabilites, axis=1)

In [14]:
prediction

array([2])

In [22]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 28, 28, 12)        108       
_________________________________________________________________
batch_normalization_20 (Batc (None, 28, 28, 12)        36        
_________________________________________________________________
activation_20 (Activation)   (None, 28, 28, 12)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 24)        10368     
_________________________________________________________________
batch_normalization_21 (Batc (None, 14, 14, 24)        72        
_________________________________________________________________
activation_21 (Activation)   (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 32)         

## Request to AI platform

In [9]:
import googleapiclient.discovery


project_id = "tensorflowdeployment" # change this to your project ID
model_id = "my_mnist_model"
model_path = "projects/{}/models/{}".format(project_id, model_id)
ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

In [16]:
# change into numpy() array 

#     input_data_json = json.dumps({
#     "signature_name": "serving_default",
#     "instances": image_np.tolist(),
# })
# request to tf serving
# SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
# response = requests.post(SERVER_URL, data=input_data_json)
# response.raise_for_status() # raise an exception in case of error
# response = response.json()
# y_proba = np.array(response["predictions"])
# get the prediction array 



input_data_json = {"signature_name": "serving_default",
                   "instances": image_np.tolist()}
request_ml = ml_resource.predict(name=model_path, body=input_data_json)
response = request_ml.execute()
if "error" in response:
    raise RuntimeError(response["error"])
y_proba = np.array([pred for pred in response["predictions"]])


In [23]:
y_proba = np.array([pred['dense_13'] for pred in response["predictions"]])


In [24]:
prediction = np.argmax(y_proba, axis=1)


In [25]:
prediction

array([2])

In [15]:
input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": image_np.tolist(),
    
})

In [16]:
import os

In [17]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


## Request to tf serving using json

In [19]:
# send request to TF serving 
import requests 

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [20]:
response

{'predictions': [[2.82079516e-11,
   9.48496e-05,
   0.995450795,
   5.35054369e-06,
   2.57140715e-11,
   8.8404413e-05,
   7.30297506e-12,
   0.00435912888,
   1.06089388e-07,
   1.4274832e-06]]}

In [24]:
# get prediction
y_proba = np.array(response["predictions"])
prediction = np.argmax(probabilites, axis=1)

In [26]:
prediction

array([2])

## Request to tf serving using GRPC API and tf serving API

gRPC (google Remote Procedure Call)

In [ ]:
# pip install tensorflow-serving-api

In [32]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
# input is a numpy array 
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(image_np))

In [40]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [41]:
response

outputs {
  key: "dense_13"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: 2.8207951599523717e-11
    float_val: 9.484960173722357e-05
    float_val: 0.9954507946968079
    float_val: 5.35054368810961e-06
    float_val: 2.5714071497096036e-11
    float_val: 8.84044129634276e-05
    float_val: 7.302975064960027e-12
    float_val: 0.004359128884971142
    float_val: 1.0608938794121059e-07
    float_val: 1.4274831983129843e-06
  }
}
model_spec {
  name: "my_mnist_model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [35]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [37]:
prediction = np.argmax(y_proba, axis=1)

In [38]:
prediction

array([2])

In [42]:
import os 
import pprint 
  
# Get the list of user's 
# environment variables 
env_var = os.environ 
  
# Print the list of user's 
# environment variables 
print("User's Environment variable:") 
pprint.pprint(dict(env_var), width = 1)

User's Environment variable:
{'CLICOLOR': '1',
 'CLUTTER_IM_MODULE': 'xim',
 'COLORTERM': 'truecolor',
 'CONDA_DEFAULT_ENV': 'cs_ftmle',
 'CONDA_EXE': '/home/coderschool/miniconda3/bin/conda',
 'CONDA_PREFIX': '/home/coderschool/miniconda3/envs/cs_ftmle',
 'CONDA_PROMPT_MODIFIER': '(cs_ftmle) ',
 'CONDA_PYTHON_EXE': '/home/coderschool/miniconda3/bin/python',
 'CONDA_SHLVL': '1',
 'CPL_ZIP_ENCODING': 'UTF-8',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1001/bus',
 'DEFAULTS_PATH': '/usr/share/gconf/ubuntu.default.path',
 'DESKTOP_AUTOSTART_ID': '10d02171fed41d3d4d157655931562855100000026870007',
 'DESKTOP_SESSION': 'ubuntu',
 'DISPLAY': ':0',
 'GDAL_DATA': '/home/coderschool/miniconda3/envs/cs_ftmle/share/gdal',
 'GDMSESSION': 'ubuntu',
 'GIT_PAGER': 'cat',
 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated',
 'GNOME_SHELL_SESSION_MODE': 'ubuntu',
 'GNOME_TERMINAL_SCREEN': '/org/gnome/Terminal/screen/819735a5_fe27_424e_9279_7a2ecd6e9167',
 'GNOME_TERMINAL_SERVICE': ':1.83',
 'GPG_AG

In [31]:
def rotate(args):
    lst, k = args
    k = k % len(lst)
    reverse(lst, 0, k - 1)
    reverse(lst, k, len(lst) - 1)
    reverse(lst, 0, len(lst) - 1)
    return lst

def reverse(lst, i, j):
    while i < j:
        
        lst[i], lst[j] = lst[j], lst[i]
        i += 1
        j -= 1

In [35]:
def reverse(lst, i, j):
    while i < j:
        print(f"i:{i}",f"j:{j}")
        lst[i], lst[j] = lst[j], lst[i]
        i += 1
        j -= 1
    return lst 

In [42]:
lst = [1, 2, 3, 4, 5, 6, 7]
k = 2 % len(lst)
k

2

In [43]:
reverse(lst, 0, k - 1)

i:0 j:1


[2, 1, 3, 4, 5, 6, 7]